In [1]:
import os
from igmapper import authorize, bold_str

In [2]:
CWD = os.getcwd()  # current working directory
SRC, LOG = f"{CWD}/src", f"{CWD}/logs"
PKL = f"{SRC}/cookies.pkl"

CHROMEDRIVER = f"{CWD}/chromedriver"
URL = "https://instagram.com/"
XPATH_CONTAINER_FEED = "//*[contains(@class, 'xw7yly9')]"

In [3]:
auth = authorize()
auth

Login • Instagram
You need to login to continue


Waiting...: 100%|██████████| 60/60 [01:00<00:00,  1.00s/it]

Cookie file founded...


'/home/lucas/Development/Side Projects/igmapper-api/src/cookies.pkl'

In [17]:
c = open(PKL, "rUt").read()
print(c)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte

In [ ]:
usr = igp.username
following_csv = f"{LOG}/{usr}_following.csv"
followers_csv = f"{LOG}/{usr}_followers.csv"
non_followers_csv = f"{LOG}/{usr}_non_followers.csv"


# Retrieve user account information

print("Retrieving user account ID...")
acc = getUserProfileInfo(usr, csrftoken)

# If the account information could not be retrieved, exit the program
if not acc:
    print("Failed to retrieve account information. Exiting.")
    exit(1)
else:
    # Save account profile information to a JSON log file
    open(f"{LOG}/{usr}-profile_info.json", "w").write(str(acc))

    # Extract user ID from the fetched account data
    user = acc["data"]["user"]
    user_id = user["id"]

bio = user["biography"]
bio_links = user["bio_links"]
category_name = user["category_name"]
follow = user["edge_follow"]["count"]
followed = user["edge_followed_by"]["count"]
full_name = user["full_name"]
is_private = user["is_private"]
is_verified = user["is_verified"]
posts_count = user["edge_owner_to_timeline_media"]["count"]

# {italic(f'https://www.instagram.com/{usr}')}
output = f"""{bold_str('Investigated profile:')}
－ {usr} {"🟓" if is_verified else ""}
－ {bold_str(posts_count)} posts     {bold_str(followed)} followers     {bold_str(follow)} following
－ {full_name if full_name else "No name"}
－ {italic(category_name) if category_name else "No category"}
－ "{bio if bio else "No bio"}"
－ {f'{[i["url"] for i in bio_links]}' if bio_links else "No links"}
－ {bold_str('This account is private') if is_private else "Public account"}"""
print(output)
# Retrieve the user's followers and following lists using the user ID
try:
    print("Get followers...")
    acc_followers = getUserFollowers(user_id, csrftoken, sessionid)
    print("Get following...")
    acc_following = getUserFollowing(user_id, csrftoken, sessionid)
except Exception as e:
    exit()
# Create dictionaries to store 'following' and 'followers' details
following_dict, followers_dict = {}, {}

# Populate the 'following' dictionary with usernames and full names
print("Building following dictionary...")
for i in acc_following:
    usr = i.get("username")
    following_dict[usr] = i.get("full_name")

# Populate the 'followers' dictionary with usernames and full names
print("Building followers dictionary...")
for i in acc_followers:
    usr = i.get("username")
    followers_dict[usr] = i.get("full_name")

# Find users that are in 'following' but not in 'followers' (people who don't follow back)
print("Get non followers...")
non_followers = [i for i in following_dict if i not in followers_dict]
csv_columns = ["Index", "Full Name", "Username", "Profile URL"]

# 1. "Não seguidores" (quem você segue, mas não te segue de volta)
if non_followers:
    if input(bold_str("Save non followers list? [Y/n]: ")).strip().lower() in ("", "y"):
        data = []
        for i, usr in enumerate(sorted(non_followers), 1):
            fullname, profile_url = following_dict[usr], f"{URL}{usr}"
            data.append([i, fullname, usr, profile_url])

        # Salvar no CSV
        save_to_csv(non_followers_csv, data, csv_columns)

if input(bold_str("Save following list? [Y/n]: ")).strip().lower() in ("", "y"):
    data = [
        [i, usr["full_name"], usr["username"], f"{URL}{usr['username']}"]
        for i, usr in enumerate(acc_following, 1)
    ]
    save_to_csv(following_csv, data, csv_columns)

# 3. Lista de "seguidores"
if input(bold_str("Save followers list? [Y/n]: ")).strip().lower() in ("", "y"):
    data = [
        [i, usr["full_name"], usr["username"], f"{URL}{usr['username']}"]
        for i, usr in enumerate(acc_followers, 1)
    ]
    save_to_csv(followers_csv, data, csv_columns)

Cookie file founded...
Retrieving user account ID...


NameError: name 'getUserProfileInfo' is not defined